# Läsa och analysera NetCDF filer med Python, Xarray och matplotlib

Detta är en kort notebook, där vi introducerar hur man kan använda Python för att läsa och analysera NetCDF-filer.

Vi kommer använda biblioteket [Xarray](https://docs.xarray.dev/en/stable/index.html) för att hantera NetCDF-data och [matplotlib](https://matplotlib.org/) och [cartopy](https://scitools.org.uk/cartopy/docs/latest/index.html) för visualiseringar.


In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray as xr

## Läsa in data

<div class="alert alert-info alert-block">
    <b>Info:</b> För att följa med i denna notebooken behöver du  <a href="https://chalmers-my.sharepoint.com/:u:/r/personal/ihannah_chalmers_se/Documents/Klimatmodellering/CMIP6%20data/2024/Project1/piControl/UK-ESM/tas_Amon_UKESM1-0-LL_piControl_r1i1p1f2_gn_196001-204912.nc?csf=1&web=1&e=s57Sm1" rel="noopener" target="_blank"><b>ladda ner denna filen</b></a> och spara den på samma plats som din kopia av den här notebooken.
</div>



In [ ]:
file = "tas_Amon_UKESM1-0-LL_piControl_r1i1p1f2_gn_196001-204912.nc"

Vi öppnar filen med funktionen `xr.open_dataset()`

In [ ]:
ds = xr.open_dataset(file)

Detta returnerar ett så kallat `Dataset`, vilket ger oss en användarvänlig representation av filens innehåll: datavariabler, koordinater, och metadata.

Exekverar vi en cell med endast variabeln `ds` returneras en interaktiv vy av datasetet.
Notera att det går att klicka på flera av ikonerna i tabellen nedan vilket visar mer av metadatan för en variabel.

In [ ]:
ds

Ovan ser vi till exempel att filen innehåller 1080 tidssteg, 144 latituder, och 192 longituder.

Datavariabeln vi är intresserade av är den som heter `tas`.

<div class="alert alert-success alert-block">
    <b>Uppgift:</b> Variabeln <code>tas</code> har ett så kallat <code>long_name</code>, vad är det?
</div>

Det finns två sätt att komma åt variabeln `tas`:

In [ ]:
ds.tas

eller

In [ ]:
ds["tas"]

`ds.tas` är en `xarray.DataArray` vilken kan liknas till en Numpy-array, men den nyttjar att dimensionerna är namngivna.
Ett dataset kan hålla flera datavariabler som delar samma koordinater t.ex. tid, latitud och longitud.

## Analysera och visualisera NetCDF-data
Xarray gör det enkelt att analysera datan in en NetCDF-fil.
Vill vi beräkna medelvärdet av en variabel använder vi metoden `.mean()`:

In [ ]:
ds.tas.mean()

Vi kan också specificera vilken dimension vi vill att medelvärdet ska beräknas över:

In [ ]:
ds.tas.mean(dim="time")

Detta kommer beräkna medelvärdet över tidsaxeln, resultatet kommer alltså innehålla medelvärdet för varje gridpunkt (latitud och longitud).

Det finns många fler inbyggda metoder t.ex. `max()`, `min()` och `std()`.
Som alltid, konsultera [dokumentationen](https://docs.xarray.dev/en/stable/index.html) och din favoritsökmotor.

Xarray gör det också enkelt att visualisera datan.
För en datavariabel finns metoden `plot`.
Använder vi `plot` direkt på en 3-dimensionell variable (tid, latitud, longitud) kommer vi få ett histogram.

In [ ]:
ds.tas.plot();

För att skapa andra plot-typer behöver vi reducera dimensionerna av vår data.
Detta kan vi göra genom att antingen beräkna t.ex. ett medelvärde över en viss dimension. 
Här beräknar vi medelvärdet av tidsdimensionen.
Plottar vi sedan detta får vi en karta med ett så kallat `pcolormesh`.

In [ ]:
ds.tas.mean(dim="time").plot();

<div class="alert alert-info">
    <b>Info:</b> Notera att xarray automatiskt lägger till x- och y-labels, en colorbar, och en titel (även om denna inte är särskilt bra).
</div>

### Indexera data
I fallet att vi är intresserade av en specifik punkt (eller slice) i vår data kan vi **indexera** likt en Numpy-array:

In [ ]:
ds.tas[5]

Här har vi valt ut det femte värdet längs den första dimensionen (tid).
Den här indexering fungerar precis som den för Numpy-array, alltså kan vi specificera fler index:

In [ ]:
ds.tas[5, 2, 0:5]

Men styrkan i xarray ligger i att kunna nyttja att dimensionerna har namn och att det är känt vad datan beskriver.
Är vi intresserade av temperaturerna för ett mindre område under 1990-talet kan vi använda metoden `sel()` för att välja ut data baserat på koordinatens värde.

In [ ]:
selection = ds.tas.sel(time="1990", lat=slice(50, 75), lon=slice(0, 22))

Jämför detta med att lista ut vilka index som data ovan ligger på.

Här specificerar vi att det ska vara en kartografisk figur genom `subplot_kw={"projection": ccrs.PlateCarree()` när vi initierar figuren.
Vi lägger sedan till ett grid med `ax.gridlines()`, och kustlinjer med `ax.coastlines()`.

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
selection.mean(dim="time").plot(ax=ax)

ax.gridlines(draw_labels=["left", "bottom"])
ax.coastlines();

ax.set_title("Average Surface temperature in 1990 for UKESM1-0-LL");

### Gruppera data

Med metoden `groupby` kan vi gruppera data baserat på dess underliggande egenskaper.
Ett bra exempel för detta är om vi vill beräkna den klimatologiska årscykeln för yttemperaturen baserat på flera år av månadsmedelvärden.

In [ ]:
ds.tas.mean(["lat", "lon"]).groupby("time.month").mean().plot();
plt.title("Average annual cycle monthly global mean temperautre");

#### Zonalt medelvärde
Att beräkna ett zonalt medelvärde innebär att man beräknar medelvärdet för varje latitud (eller längs med longituden).
Kombinerar vi detta med att gruppera månaderna och beräkna medelvärdet kan vi skapa ett så kallat [Hovmöller-diagram](https://en.wikipedia.org/wiki/Hovm%C3%B6ller_diagram).

In [ ]:
fig, ax = plt.subplots()
ds.tas.mean(dim="lon").groupby("time.month").mean().plot(ax=ax, yincrease=False);
ax.set_title("Zonal mean surface temperature");

# Nästa steg
Fortsätt med [Övning 5](02_exercise_5.ipynb).